In [ ]:
import math
import time

import bokeh
import numpy as np
from bokeh.io import output_notebook, push_notebook, show
from bokeh.models import DatetimeTickFormatter, HoverTool
from bokeh.plotting import figure

In [ ]:
%load_ext autoreload

In [ ]:
output_notebook()

In [ ]:
def gen_plot(source):
    plot = bokeh.plotting.figure(
        plot_height=300, plot_width=800, background_fill_color="lightgrey", title="", y_range=(-1.1, 1.1)
    )
    plot.circle("x", "y", source=source)

    plot.x_range.follow = "end"
    plot.x_range.follow_interval = 100000

    plot.xaxis.major_label_orientation = math.pi / 4
    plot.xaxis.formatter = DatetimeTickFormatter(
        milliseconds=["%H:%M:%S"], seconds=["%H:%M:%S"], minsec=["%H:%M:%S"], minutes=["%H:%M:%S"]
    )
    return plot

In [ ]:
output_notebook()

source = bokeh.models.ColumnDataSource({"x": [], "y": []})

plot = gen_plot(source)

handle = bokeh.io.show(plot, notebook_handle=True)

In [ ]:
# output_notebook()

source2 = bokeh.models.ColumnDataSource({"x": [], "y": []})

plot2 = gen_plot(source2)

handle2 = bokeh.io.show(plot2, notebook_handle=True)

In [ ]:
for x in range(10000):
    y = np.sin(x / 10)
    source.stream({"x": [x], "y": [y]}, rollover=20)
    source2.stream({"x": [x], "y": [y * 0.5]}, rollover=200)
    bokeh.io.push_notebook(handle=handle)
    bokeh.io.push_notebook(handle=handle2)
    time.sleep(0.05)

In [ ]:
class MockCache:
    
    def read(self, *args, **kwargs):
        for x in range(10000):
            y = np.sin(x / 10)
            yield {"x": [x], "y": [y]}
            time.sleep(0.1)

In [ ]:
from apache_beam.runners.interactive.display import data_server

In [ ]:
column_source_kwargs = {"rollover": 100}

In [ ]:
sp = data_server.StreamingPlot(MockCache(), lambda x: x, gen_plot, **column_source_kwargs)

In [ ]:
sp.start()

In [ ]:
sp.stop()

In [ ]:
del sp

In [ ]:
sp = data_server.StreamingPlot(MockCache(), lambda x: x, gen_plot, source_type="ajax")

In [ ]:
sp.start()

In [ ]:
del sp